This notebook shows how to use stamp.py in a notebook or interactive setting.  I recommend command line usage for large catalogs and/or large numbers of images.  The catalog must be formatted as shown in this notebook.

First, import stamp:

In [3]:
import stamp

/Users/kpenner/anaconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


stamp.py has 3 main functions: cutout, measure_rms, and output eps.

The first step is to use cutout, which takes the images, positions from a catalog, and target names, to create the FITS cutouts.

Now, read in the catalog and any number of images:

In [14]:
from astropy.table import Table
from astropy.io import fits
from astropy import units as u

In [5]:
cat = Table.read('../ian_cats/VLA-HFF_0416_compact_optical_rasort.txt',
                 format = 'ascii.sextractor')

In [7]:
img_list = ['../radio_imgs/img_MACSJ0416_S-BAND_data-MFS-image.fits',
            '../radio_imgs/img_MACSJ0416_C-BAND_data-MFS-image.fits',
            '../hst_imgs/macs0416/hlsp_frontier_hst_acs-60mas-selfcal_macs0416_f435w_v1.0_drz.fits',
            '../hst_imgs/macs0416/hlsp_frontier_hst_acs-60mas-selfcal_macs0416_f606w_v1.0_drz.fits',
            '../hst_imgs/macs0416/hlsp_frontier_hst_wfc3-60mas-bkgdcor_macs0416_f160w_v1.0-epoch2_drz.fits',
            '../hst_imgs/macs0416/parallel/hlsp_frontier_hst_acs-60mas-selfcal_macs0416-hffpar_f435w_v1.0-epoch2_drz.fits',
            '../hst_imgs/macs0416/parallel/hlsp_frontier_hst_acs-60mas-selfcal_macs0416-hffpar_f606w_v1.0-epoch2_drz.fits',
            '../hst_imgs/macs0416/parallel/hlsp_frontier_hst_wfc3-60mas_macs0416-hffpar_f160w_v1.0_drz.fits']

In [8]:
imgs = [fits.open(img) for img in img_list]

In [9]:
imgs

[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb130d8b50>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb1300a450>, <astropy.io.fits.hdu.table.BinTableHDU object at 0xb136b0490>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb13242ad0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0xb1370e550>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb1332b610>, <astropy.io.fits.hdu.table.BinTableHDU object at 0xb13764350>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb13436c90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0xb1379ae50>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb13541a10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0xb137f0110>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0xb136320d0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0xb1383bed0>]]

In [20]:
cat[0:5]

NAME,RA,DEC,SIGMA_RA,SIGMA_DEC,INT_FLUX_S,SIGMA_INT_FLUX_S,PEAK_FLUX_S,SIGMA_PEAK_FLUX_S,BEST_FLUX_S,SIGMA_BEST_FLUX_S,PB_S,INT_FLUX_C,SIGMA_INT_FLUX_C,PEAK_FLUX_C,SIGMA_PEAK_FLUX_C,BEST_FLUX_C,SIGMA_BEST_FLUX_C,PB_C,ALPHA,SIGMA_ALPHA,IM_MAJ,SIGMA_IM_MAJ,IM_MIN,SIGMA_IM_MIN,IM_PA,SIGMA_IM_PA,DC_MAJ,SIGMA_DC_MAJ,DC_MIN,SIGMA_DC_MIN,DC_PA,SIGMA_DC_PA,RESOLVED_FLAG,ASTRODEEP_ID,ASTRODEEP_RA,ASTRODEEP_DEC,ASTRODEEP_SEP,MAG_JH140,MAGERR_JH140,ZBEST,ZBEST_SIQR,MAGNIF,MSTAR,MSTAR_MIN,MSTAR_MAX,SFR,SFR_MIN,SFR_MAX,RELFLAG,SUBARU_ID,SUBARU_RA,SUBARU_DEC,SUBARU_SEP,ZMAG,DZMAG,ZBPZ,ZBPZ_LOWER,ZBPZ_UPPER,MU_LOWER,MU_MEDIAN,MU_UPPER,MU_STD,MU_MAD,N_MODELS
str26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,str4,str6,str4,str6,str4,str4,str5,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,str8,str9,str4,str6,str6,str4,str4,str4,str6,str6,str6,str6,str5,str7,str1,str5,str8,str9,str4,str5,str4,str4,str4,str4,str4,str4,str4,str4,str4,str4
VLAHFF-J041523.37-240205.2,63.84739,-24.0348,0.05,0.08,72.11,6.97,33.73,5.0,72.11,6.97,0.21,-,-,-,-,-,-,-,-,-,1.14,0.18,0.89,0.12,168.84,28.98,0.79,0.18,0.58,0.12,117.49,28.98,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,60765,63.84732,-24.03475,0.32,24.16,0.03,1.42,0.87,1.94,1.10,1.13,1.15,0.00,0.00,1.00
VLAHFF-J041529.21-240751.4,63.87175,-24.13097,0.01,0.03,107.17,5.51,76.86,3.41,107.17,5.51,0.29,-,-,-,-,-,-,-,-,-,1.1,0.06,0.61,0.02,5.61,4.08,0.58,0.06,0.32,0.02,15.51,4.08,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,40926,63.87179,-24.13102,0.24,18.59,0.00,0.49,0.26,0.55,1.00,1.05,1.08,0.00,0.00,1.00
VLAHFF-J041533.34-240730.5,63.88895,-24.12515,0.0,0.01,143.39,4.46,137.47,2.48,143.39,4.46,0.38,-,-,-,-,-,-,-,-,-,0.94,0.02,0.53,0.01,3.21,1.67,0.17,0.02,0.0,0.01,66.79,1.67,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,40866,63.88893,-24.12515,0.05,22.42,0.01,1.14,0.95,3.26,1.15,1.17,1.23,0.00,0.00,1.00
VLAHFF-J041533.68-240824.3,63.89035,-24.14009,0.01,0.04,46.03,4.71,40.85,2.66,43.37,2.63,0.35,-,-,-,-,-,-,-,-,-,1.02,0.09,0.52,0.02,0.52,5.27,0.41,0.09,0.13,0.02,174.36,5.27,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,38162,63.89034,-24.14009,0.05,20.63,0.00,0.73,0.69,0.75,1.06,1.06,1.06,0.00,0.00,1.00
VLAHFF-J041534.50-240016.1,63.89379,-24.00448,0.03,0.07,31.37,4.48,23.28,2.71,27.02,2.49,0.36,-,-,-,-,-,-,-,-,-,1.12,0.17,0.57,0.05,12.0,9.59,0.65,0.17,0.14,0.05,27.98,9.59,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,67484,63.89375,-24.00445,0.18,23.06,0.02,0.50,0.47,3.56,1.01,1.01,1.05,0.00,0.00,1.00


We are going to use the NAME field (which must be called NAME if stamp is used at the command line) as the target name.  I have modified the header of Ian's catalogs; instead of starting at "0 NAME", it starts at "1 NAME", the SExtractor default.

We also need the RA and DEC fields (which must be called RA and DEC is stamp is used at the command line).

In [15]:
targname = cat['NAME'].astype(str)
ra = cat['RA']*u.degree
dec = cat['DEC']*u.degree

Now, call cutout:

In [22]:
stamp.cutout(imgs, ra, dec, targname)

For each entry in the catalog, cutout attempts to make a stamp for each image.  If an image has no coverage for the source, the stamp is not written.

The files are numbered first by their row in the catalog.  0img0.fits is the stamp for the first entry and the first image.  Images without a FILTER keyword in the FITS header are sorted after the first image.  The exception is an image with TELESCOP = 'Spitzer' and CHNLNUM = 1 in the FITS header.  This image, and images with select HST FILTER keywords, are sorted by increasing wavelength.  E.g., for this exercise,

[n]img0.fits = S band stamp

[n]img1.fits = C band stamp

[n]img2.fits = F435W stamp, main observation

[n]img3.fits = F606W stamp, main observation

[n]img4.fits = F160W stamp, main observation

[n]img5.fits = F435W stamp, parallel observation

[n]img6.fits = F606W stamp, parallel observation

[n]img7.fits = F160W stamp, parallel observation

[n]img8.fits = IRAC channel 1 stamp